<a href="https://colab.research.google.com/github/nattatorn-chu/HAREC-IOT/blob/main/Source%20Code/KNN%20Multi-Output%20Classification%20%5BCompare%200%20vs%20Drop%20Activity%20Feature%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


In [3]:
df_init = pd.read_csv('../content/drive/MyDrive/ExtraSensory/combine-6-activity/raw-data.csv')
df_init.head()

,raw_acc:3d:mean_x,raw_acc:3d:mean_y,raw_acc:3d:mean_z,raw_acc:3d:std_x,raw_acc:3d:std_y,raw_acc:3d:std_z,proc_gyro:3d:mean_x,proc_gyro:3d:mean_y,proc_gyro:3d:mean_z,proc_gyro:3d:std_x,proc_gyro:3d:std_y,proc_gyro:3d:std_z,label:LOC_main_workplace,label:LOC_home,label:AT_SCHOOL,label:LYING_DOWN,label:SITTING,label:SLEEPING
0,-0.020237,0.000716,1.000499,0.004348,0.004912,0.002900,-0.006901,-0.011557,-0.009601,0.001290,0.001219,0.001406,0,0,1,0,1,0
1,-0.020832,0.001917,1.001211,0.001971,0.002744,0.002017,-0.007043,-0.011394,-0.009715,0.001127,0.001117,0.001077,0,0,1,0,1,0
2,-0.020724,0.002158,1.001293,0.002049,0.002378,0.003091,-0.006606,-0.011399,-0.009501,0.001148,0.004607,0.001202,0,0,1,0,1,0
3,-0.020495,0.001989,1.000751,0.002525,0.003532,0.001959,-0.006365,-0.011673,-0.009756,0.001146,0.001120,0.001338,0,0,1,0,1,0
4,-0.019733,0.001100,1.000929,0.005440,0.003670,0.025675,-0.006534,-0.011633,-0.009719,0.005265,0.046299,0.001181,0,0,1,0,1,0


# **K-Nearest Neighbors (KNN) classifier with Multi-Output Classification**

In [4]:
# Load the data from CSV file
data = pd.read_csv('../content/drive/MyDrive/ExtraSensory/combine-6-activity/raw-data.csv')

# Define the label names
labels = ['label:LOC_main_workplace','label:LOC_home','label:AT_SCHOOL','label:LYING_DOWN','label:SITTING','label:SLEEPING']

# Split the data into training and testing sets
X = data.iloc[:, :12]
y = data.iloc[:, 12:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the model on the training data
k = 3
clf = MultiOutputClassifier(KNeighborsClassifier(n_neighbors=k))
clf.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = clf.predict(X_test)

# Evaluate the model's performance
report = classification_report(y_test, y_pred, target_names=labels)
accuracy = clf.score(X_test, y_test)

# Add the confusion matrix and TP/TN/FP/FN output
cm = confusion_matrix(y_test.values.argmax(axis=1), y_pred.argmax(axis=1))
TP = cm[1, 1]
TN = cm[0, 0]
FP = cm[0, 1]
FN = cm[1, 0]
# print("Confusion Matrix:")
# print(cm)
print(report)
print("-----------------------------------------------------------------------")
print("Accuracy:", accuracy)
print("True Positives:", TP)
print("True Negatives:", TN)
print("False Positives:", FP)
print("False Negatives:", FN)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                          precision    recall  f1-score   support

label:LOC_main_workplace       0.80      0.75      0.77      3807
          label:LOC_home       0.90      0.88      0.89      9186
         label:AT_SCHOOL       0.78      0.72      0.75      3993
        label:LYING_DOWN       0.95      0.93      0.94      6808
           label:SITTING       0.87      0.87      0.87      9499
          label:SLEEPING       0.96      0.95      0.95      5678

               micro avg       0.89      0.87      0.88     38971
               macro avg       0.88      0.85      0.86     38971
            weighted avg       0.89      0.87      0.88     38971
             samples avg       0.77      0.77      0.76     38971

-----------------------------------------------------------------------
Accuracy: 0.7447478991596639
True Positives: 7922
True Negatives: 4458
False Positives: 401
False Negatives: 492


# **Set the specified sensor values to 0**

In [5]:
# Load the CSV file into a pandas dataframe
df = pd.read_csv('../content/drive/MyDrive/ExtraSensory/combine-6-activity/raw-data.csv')

# Dictionary of activity to sensor value mappings
activity_sensor_map = {
    'label:LOC_main_workplace': ['raw_acc:3d:std_x', 'proc_gyro:3d:mean_x', 'proc_gyro:3d:mean_z'],
    'label:LOC_home': ['raw_acc:3d:std_x', 'raw_acc:3d:std_y', 'proc_gyro:3d:mean_y'],
    'label:AT_SCHOOL': ['raw_acc:3d:std_x', 'raw_acc:3d:std_z', 'proc_gyro:3d:mean_z'],
    'label:LYING_DOWN': ['raw_acc:3d:std_x', 'raw_acc:3d:std_y', 'proc_gyro:3d:mean_y'],
    'label:SITTING': ['raw_acc:3d:std_y', 'proc_gyro:3d:mean_z', 'proc_gyro:3d:std_x'],
    'label:SLEEPING': ['raw_acc:3d:std_x', 'raw_acc:3d:std_y', 'proc_gyro:3d:mean_y']
}

# Loop over each activity and modify the specified sensor values
for activity, sensor_values in activity_sensor_map.items():
    # Select only the rows where the activity is 1
    activity_df = df[df[activity] == 1]
    
    # Set the specified sensor values to 0
    activity_df.loc[:, sensor_values] = 0
    
    # Merge the modified activity_df back into the original dataframe
    df.update(activity_df)

# Save the modified dataframe back to a CSV file
df.to_csv("../content/drive/MyDrive/ExtraSensory/combine-6-activity/modified_activity/data_set_zero.csv", index=False)

<ipython-input-5-8c859ce65daa>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activity_df.loc[:, sensor_values] = 0
<ipython-input-5-8c859ce65daa>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activity_df.loc[:, sensor_values] = 0
<ipython-input-5-8c859ce65daa>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activity_df.loc[:, sensor_values] = 0
<ipython-input-5-8c859ce6

In [6]:
df_setzero = pd.read_csv('../content/drive/MyDrive/ExtraSensory/combine-6-activity/modified_activity/data_set_zero.csv')
df_setzero.head()

,raw_acc:3d:mean_x,raw_acc:3d:mean_y,raw_acc:3d:mean_z,raw_acc:3d:std_x,raw_acc:3d:std_y,raw_acc:3d:std_z,proc_gyro:3d:mean_x,proc_gyro:3d:mean_y,proc_gyro:3d:mean_z,proc_gyro:3d:std_x,proc_gyro:3d:std_y,proc_gyro:3d:std_z,label:LOC_main_workplace,label:LOC_home,label:AT_SCHOOL,label:LYING_DOWN,label:SITTING,label:SLEEPING
0,-0.020237,0.000716,1.000499,0.0,0.0,0.0,-0.006901,-0.011557,0.0,0.0,0.001219,0.001406,0.0,0.0,1.0,0.0,1.0,0.0
1,-0.020832,0.001917,1.001211,0.0,0.0,0.0,-0.007043,-0.011394,0.0,0.0,0.001117,0.001077,0.0,0.0,1.0,0.0,1.0,0.0
2,-0.020724,0.002158,1.001293,0.0,0.0,0.0,-0.006606,-0.011399,0.0,0.0,0.004607,0.001202,0.0,0.0,1.0,0.0,1.0,0.0
3,-0.020495,0.001989,1.000751,0.0,0.0,0.0,-0.006365,-0.011673,0.0,0.0,0.001120,0.001338,0.0,0.0,1.0,0.0,1.0,0.0
4,-0.019733,0.001100,1.000929,0.0,0.0,0.0,-0.006534,-0.011633,0.0,0.0,0.046299,0.001181,0.0,0.0,1.0,0.0,1.0,0.0


In [7]:
# Load the data from CSV file
df_setzero = pd.read_csv('../content/drive/MyDrive/ExtraSensory/combine-6-activity/modified_activity/data_set_zero.csv')

# Define the label names
labels = ['label:LOC_main_workplace','label:LOC_home','label:AT_SCHOOL','label:LYING_DOWN','label:SITTING','label:SLEEPING']

# Split the df_setzero into training and testing sets
X = df_setzero.iloc[:, :12]
y = df_setzero.iloc[:, 12:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the model on the training df_setzero
k = 3
clf = MultiOutputClassifier(KNeighborsClassifier(n_neighbors=k))
clf.fit(X_train, y_train)

# Make predictions on the testing df_setzero
y_pred = clf.predict(X_test)

# Evaluate the model's performance
report = classification_report(y_test, y_pred, target_names=labels)
accuracy = clf.score(X_test, y_test)

# Add the confusion matrix and TP/TN/FP/FN output
cm = confusion_matrix(y_test.values.argmax(axis=1), y_pred.argmax(axis=1))
TP = cm[1, 1]
TN = cm[0, 0]
FP = cm[0, 1]
FN = cm[1, 0]
# print("Confusion Matrix:")
# print(cm)
print(report)
print("-----------------------------------------------------------------------")
print("Accuracy:", accuracy)
print("True Positives:", TP)
print("True Negatives:", TN)
print("False Positives:", FP)
print("False Negatives:", FN)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                          precision    recall  f1-score   support

label:LOC_main_workplace       0.85      0.83      0.84      3807
          label:LOC_home       0.93      0.93      0.93      9186
         label:AT_SCHOOL       0.90      0.88      0.89      3993
        label:LYING_DOWN       0.96      0.95      0.96      6808
           label:SITTING       0.96      0.98      0.97      9499
          label:SLEEPING       0.96      0.96      0.96      5678

               micro avg       0.94      0.93      0.94     38971
               macro avg       0.93      0.92      0.92     38971
            weighted avg       0.94      0.93      0.94     38971
             samples avg       0.85      0.85      0.84     38971

-----------------------------------------------------------------------
Accuracy: 0.8519385026737968
True Positives: 8376
True Negatives: 4938
False Positives: 268
False Negatives: 289


# **Drop the specified sensor columns**

In [8]:
# Load the CSV file into a pandas dataframe
df = pd.read_csv('../content/drive/MyDrive/ExtraSensory/combine-6-activity/raw-data.csv')

# Dictionary of activity to sensor value mappings
activity_sensor_map = {
    'label:LOC_main_workplace': ['raw_acc:3d:std_x', 'proc_gyro:3d:mean_x', 'proc_gyro:3d:mean_z'],
    'label:LOC_home': ['raw_acc:3d:std_x', 'raw_acc:3d:std_y', 'proc_gyro:3d:mean_y'],
    'label:AT_SCHOOL': ['raw_acc:3d:std_x', 'raw_acc:3d:std_z', 'proc_gyro:3d:mean_z'],
    'label:LYING_DOWN': ['raw_acc:3d:std_x', 'raw_acc:3d:std_y', 'proc_gyro:3d:mean_y'],
    'label:SITTING': ['raw_acc:3d:std_y', 'proc_gyro:3d:mean_z', 'proc_gyro:3d:std_x'],
    'label:SLEEPING': ['raw_acc:3d:std_x', 'raw_acc:3d:std_y', 'proc_gyro:3d:mean_y']
}

# Loop over each activity and drop the specified columns
for activity, sensor_values in activity_sensor_map.items():
    # Select only the rows where the activity is 1
    activity_df = df[df[activity] == 1]
    
    # Drop the specified columns
    activity_df.loc[:, sensor_values] = ''
    # activity_df.drop(columns=sensor_values, inplace=True)
    
    # Merge the modified activity_df back into the original dataframe
    df.update(activity_df)

# Save the modified dataframe back to a CSV file
df.to_csv('../content/drive/MyDrive/ExtraSensory/combine-6-activity/modified_activity/data_set_drop.csv', index=False)

<ipython-input-8-486b9b040de6>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activity_df.loc[:, sensor_values] = ''
<ipython-input-8-486b9b040de6>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activity_df.loc[:, sensor_values] = ''
<ipython-input-8-486b9b040de6>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [9]:
df = pd.read_csv('../content/drive/MyDrive/ExtraSensory/combine-6-activity/raw-data.csv')
df.head()

,raw_acc:3d:mean_x,raw_acc:3d:mean_y,raw_acc:3d:mean_z,raw_acc:3d:std_x,raw_acc:3d:std_y,raw_acc:3d:std_z,proc_gyro:3d:mean_x,proc_gyro:3d:mean_y,proc_gyro:3d:mean_z,proc_gyro:3d:std_x,proc_gyro:3d:std_y,proc_gyro:3d:std_z,label:LOC_main_workplace,label:LOC_home,label:AT_SCHOOL,label:LYING_DOWN,label:SITTING,label:SLEEPING
0,-0.020237,0.000716,1.000499,0.004348,0.004912,0.002900,-0.006901,-0.011557,-0.009601,0.001290,0.001219,0.001406,0,0,1,0,1,0
1,-0.020832,0.001917,1.001211,0.001971,0.002744,0.002017,-0.007043,-0.011394,-0.009715,0.001127,0.001117,0.001077,0,0,1,0,1,0
2,-0.020724,0.002158,1.001293,0.002049,0.002378,0.003091,-0.006606,-0.011399,-0.009501,0.001148,0.004607,0.001202,0,0,1,0,1,0
3,-0.020495,0.001989,1.000751,0.002525,0.003532,0.001959,-0.006365,-0.011673,-0.009756,0.001146,0.001120,0.001338,0,0,1,0,1,0
4,-0.019733,0.001100,1.000929,0.005440,0.003670,0.025675,-0.006534,-0.011633,-0.009719,0.005265,0.046299,0.001181,0,0,1,0,1,0


In [10]:
df_dropfeature = pd.read_csv('../content/drive/MyDrive/ExtraSensory/combine-6-activity/modified_activity/data_set_drop.csv')
df_dropfeature.head()

,raw_acc:3d:mean_x,raw_acc:3d:mean_y,raw_acc:3d:mean_z,raw_acc:3d:std_x,raw_acc:3d:std_y,raw_acc:3d:std_z,proc_gyro:3d:mean_x,proc_gyro:3d:mean_y,proc_gyro:3d:mean_z,proc_gyro:3d:std_x,proc_gyro:3d:std_y,proc_gyro:3d:std_z,label:LOC_main_workplace,label:LOC_home,label:AT_SCHOOL,label:LYING_DOWN,label:SITTING,label:SLEEPING
0,-0.020237,0.000716,1.000499,NaN,NaN,NaN,-0.006901,-0.011557,NaN,NaN,0.001219,0.001406,0.0,0.0,1.0,0.0,1.0,0.0
1,-0.020832,0.001917,1.001211,NaN,NaN,NaN,-0.007043,-0.011394,NaN,NaN,0.001117,0.001077,0.0,0.0,1.0,0.0,1.0,0.0
2,-0.020724,0.002158,1.001293,NaN,NaN,NaN,-0.006606,-0.011399,NaN,NaN,0.004607,0.001202,0.0,0.0,1.0,0.0,1.0,0.0
3,-0.020495,0.001989,1.000751,NaN,NaN,NaN,-0.006365,-0.011673,NaN,NaN,0.001120,0.001338,0.0,0.0,1.0,0.0,1.0,0.0
4,-0.019733,0.001100,1.000929,NaN,NaN,NaN,-0.006534,-0.011633,NaN,NaN,0.046299,0.001181,0.0,0.0,1.0,0.0,1.0,0.0


In [11]:
# Data Frame Drop
df_dropfeature = pd.read_csv('../content/drive/MyDrive/ExtraSensory/combine-6-activity/modified_activity/data_set_drop.csv')
df_dropfeature = df_dropfeature.dropna(axis=1)
df_dropfeature.to_csv('../content/drive/MyDrive/ExtraSensory/combine-6-activity/modified_activity/data_set_drop.csv', index=False)

In [12]:
df_dropfeature = pd.read_csv('../content/drive/MyDrive/ExtraSensory/combine-6-activity/modified_activity/data_set_drop.csv')
df_dropfeature.head()

,raw_acc:3d:mean_x,raw_acc:3d:mean_y,raw_acc:3d:mean_z,proc_gyro:3d:std_y,proc_gyro:3d:std_z,label:LOC_main_workplace,label:LOC_home,label:AT_SCHOOL,label:LYING_DOWN,label:SITTING,label:SLEEPING
0,-0.020237,0.000716,1.000499,0.001219,0.001406,0.0,0.0,1.0,0.0,1.0,0.0
1,-0.020832,0.001917,1.001211,0.001117,0.001077,0.0,0.0,1.0,0.0,1.0,0.0
2,-0.020724,0.002158,1.001293,0.004607,0.001202,0.0,0.0,1.0,0.0,1.0,0.0
3,-0.020495,0.001989,1.000751,0.001120,0.001338,0.0,0.0,1.0,0.0,1.0,0.0
4,-0.019733,0.001100,1.000929,0.046299,0.001181,0.0,0.0,1.0,0.0,1.0,0.0


In [14]:
# Load the df_dropfeature from CSV file
df_dropfeature = pd.read_csv('../content/drive/MyDrive/ExtraSensory/combine-6-activity/modified_activity/data_set_drop.csv')

# Define the label names
labels = ['label:LOC_main_workplace','label:LOC_home','label:AT_SCHOOL','label:LYING_DOWN','label:SITTING','label:SLEEPING']

# Split the df_dropfeature into training and testing sets
X = df_dropfeature.iloc[:, :5]
y = df_dropfeature.iloc[:, 5:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the model on the training df_dropfeature
k = 3
clf = MultiOutputClassifier(KNeighborsClassifier(n_neighbors=k))
clf.fit(X_train, y_train)

# Make predictions on the testing df_dropfeature
y_pred = clf.predict(X_test)

# Evaluate the model's performance
report = classification_report(y_test, y_pred, target_names=labels)
accuracy = clf.score(X_test, y_test)

# Add the confusion matrix and TP/TN/FP/FN output
cm = confusion_matrix(y_test.values.argmax(axis=1), y_pred.argmax(axis=1))
TP = cm[1, 1]
TN = cm[0, 0]
FP = cm[0, 1]
FN = cm[1, 0]
# print("Confusion Matrix:")
# print(cm)
print(report)
print("-----------------------------------------------------------------------")
print("Accuracy:", accuracy)
print("True Positives:", TP)
print("True Negatives:", TN)
print("False Positives:", FP)
print("False Negatives:", FN)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                          precision    recall  f1-score   support

label:LOC_main_workplace       0.79      0.73      0.76      3807
          label:LOC_home       0.90      0.87      0.89      9186
         label:AT_SCHOOL       0.77      0.71      0.74      3993
        label:LYING_DOWN       0.95      0.93      0.94      6808
           label:SITTING       0.86      0.86      0.86      9499
          label:SLEEPING       0.96      0.95      0.95      5678

               micro avg       0.89      0.86      0.87     38971
               macro avg       0.87      0.84      0.86     38971
            weighted avg       0.88      0.86      0.87     38971
             samples avg       0.76      0.76      0.75     38971

-----------------------------------------------------------------------
Accuracy: 0.7321428571428571
True Positives: 7849
True Negatives: 4403
False Positives: 374
False Negatives: 526
